# Trial Run for the BranchPro module using Synthetic Data

In order to assess how well the functionalities implemented in the ``branchpro`` module cope with realistic data, we will first generate some artificial incidence cases using a reasonable instance of an R profile.


In [1]:
# Import requirements
import branchpro

import numpy as np
import pandas as pd
import scipy

## Set true R profile

In [2]:
# Read the reproduction number values
new_rs = [3, 2.7, 2.5, 2.4, 2.3, 2.2, 1.9, 1.8, 1.9, 1.7, 1.6, 1.3, 1.3, 1.2, 1.1, 0.9, 1, 1.1, 0.95, 0.9, 0.8, 0.7, 0.75, 0.8, 0.77, 0.73, 0.7, 0.68, 0.65, 0.67, 0.66, 0.72]

# Construct vector of the times at which each of the R number changes occur
start_times = np.arange(1, len(new_rs)+1, dtype=int)

## Generate Incidence Data

Using this realistically behaved reproduction numbers, we will use the forward simulation functionality to sample incidence data which we will use in the following steps to do inference and try to reconstruct the true R profile underlining the data.

# Set evaluation times and serial interval distribution

We wish to obtain data for all the time points for which we know the value of the *reproduction number*. We also assume that the *serial interval* is shaped similarly to a gamma distribution, which we will in turn discretize to fit the way the *forward simulation* works.

In [3]:
# Take evaluation times for the forward simulation as the times corresponding to the R profile
times = start_times

# Build the serial interval w_s
num_timepoints = len(new_rs)
ws_mean = 2.6
ws_var = 1.5**2
theta = ws_var / ws_mean
k = ws_mean / theta
 
w_dist = scipy.stats.gamma(k, scale=theta)
disc_w = w_dist.pdf(np.arange(num_timepoints))

We assume that the imported cases are sampled from a zero-inflated poisson with given mean ($\lambda$) and weight ($\pi$). For this we will cosntruct first a function to sample observations from such a distribution.

In [4]:
# Construct a zero-inflated poisson sampler
def zero_inf_poss_sampler(pi, mean, num_samples=1):
    # First check if we sample from the Poisson or Dirac centred at the origin distribution
    unif_samples = scipy.stats.uniform.rvs(size=num_samples)
    sample = np.zeros_like(unif_samples)

    for s in range(num_samples):
        # If we fall out of the Dirac category, we sample from Poisson
        if unif_samples[s] > pi:
            sample[s] = scipy.stats.poisson.rvs(mean)
    
    return sample

In [8]:
# Build the imported cases
pi = 0.6
ic_mean = 10
imported_times = np.arange(1,(num_timepoints+1))

imported_cases = zero_inf_poss_sampler(pi=pi, mean=ic_mean, num_samples=num_timepoints)